In [ ]:
from OSMPythonTools.api import Api
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

In [ ]:
nominatim = Nominatim()
cracow = nominatim.query('Kraków, Poland')
city = nominatim.query('Małopolska, Poland')

In [ ]:
selectors = [['"tourism"="museum"'],
            ['"tourism"="gallery"'],
             ['"tourism"="aquarium"'],
             ['"tourism"="artwork"'],
             ['"tourism"="theme_park"'],
             ['"tourism"="viewpoint"'],
             ['"tourism"="zoo"'],
             ['"heritage"="2"'],
             ['"historic"="building"'],
             ['"historic"="castle"'],
             ['"historic"="city_gate"'],
             ['"leisure"="firepit"'],
             ['"leisure"="fishing"'],
             ['"leisure"="garden"'],
             ['"leisure"="water_park"'],
             ['"memorial"="statue"'],
             ['"military"="bunker"'],
             ['"natural"="peak"']]
overpass = Overpass()
query = overpassQueryBuilder(area=city.areaId(), elementType=['node', 'way'], selector=selectors[0], out='body')
res = overpass.query(query)


In [ ]:
print(list(map(lambda x: x.tag('name'), res.elements())))


In [ ]:
res.toJSON()['elements'][0]

In [ ]:
overpass = Overpass()
query = overpassQueryBuilder(area=city.areaId(), elementType=['node', 'way'], selector=['"religion"~".*"'], out='body')
res = overpass.query(query)
print(list(map(lambda x: x.tag('name'), res.elements())))


In [ ]:
print(list(map(lambda x: x.tag('opening_hours'), res.elements())))


In [ ]:
print(res.elements()[0].type())
print(res.elements()[0].id())
print(res.elements()[0].tags())


In [ ]:
api = Api()

node = api.query('way/85151350')
print(node.tags())


In [ ]:
from pyvis.network import Network

In [ ]:
def show_graph():
    N = Network(notebook=True)
    for cat in range(len(categories)):
        N.add_node(cat, categories[cat].get("name"))
        sub = categories[cat].get("sub_categories")
        for sub_cat in range(len(sub)):
            N.add_node(sub_cat, sub[sub_cat].get("name"))

    for cat in range(len(categories)):
        sub = categories[cat].get("sub_categories")
        for sub_cat in range(len(sub)):
            N.add_edge(cat, sub_cat, weight=1)
        for other in range(len(categories)):
            if cat != other:
                N.add_edge(cat, other, weight=5)

    N.toggle_physics(True)
    N.show("categories_graph.html")


In [ ]:
from category import categories
from models.mongo_utils import MongoUtils

print(categories)

In [ ]:
client: MongoUtils = MongoUtils()

db = client.get_db()
collection = db["categories-graph"]
collection2 = db["categories"]

max_sub = 0
for cat in categories:
    if cat.get("sub_categories") is None:
        print(f"A category {cat.get('name')} does not have subcategories")
        continue
    if len(cat.get("sub_categories")) > max_sub:
        max_sub = len(cat.get("sub_categories"))

max_sub += 1
for i in range(len(categories)):
    cat = categories[i]
    if cat.get('additional_codes') is not None:
        collection2.insert_one({
            'id': i*max_sub,
            'name': cat.get('name'),
            'code': cat.get('code'),
            'visiting_time': cat.get("visiting_time"),
            'additional_codes': cat.get('additional_codes')})
    else:
        collection2.insert_one({
            'id': i*max_sub,
            'name': cat.get('name'),
            'code': cat.get('code'),
            'visiting_time': cat.get("visiting_time")})

    sub = cat.get("sub_categories")
    if cat.get("sub_categories") is None: continue

    for j in range(len(sub)):
        sub_cat = sub[j]
        if sub_cat.get('additional_codes') is not None:
            collection2.insert_one({
                'id': i*max_sub+j+1,
                'name': sub_cat.get('name'),
                'code': sub_cat.get('code'),
                'visiting_time': cat.get("visiting_time"),
                'additional_codes': sub_cat.get('additional_codes')})
        else:
            collection2.insert_one({
                'id': i*max_sub+j+1,
                'name': sub_cat.get('name'),
                'code': sub_cat.get('code'),
                'visiting_time': cat.get("visiting_time"),})

for cat in range(len(categories)):
    sub = categories[cat].get("sub_categories")
    for other in range(cat+1, len(categories)):
        collection.insert_one({'from': categories[cat].get("name"), 'from_id': cat*max_sub,
                               'to':categories[other].get("name"), 'to_id': other*max_sub,
                               'weight': 5})
    if sub is None: continue
    for sub_cat in range(len(sub)):
        collection.insert_one({'from': categories[cat].get("name"), 'from_id': cat*max_sub,
                               'to':sub[sub_cat].get("name"), 'to_id': cat*max_sub + sub_cat + 1,
                               'weight': 1})



In [ ]:
from mongo_utils import MongoUtils

In [ ]:
db = MongoUtils()
v2 = db.get_collection('cracow-attractions-v2').find()
v3 = db.get_collection('cracow-attractions-v3')

xids = []
for i in v2:
    if i['xid'] not in xids:
        pass# v3.insert_one(i)
    xids.append(i['xid'])

In [ ]:
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

In [ ]:
overpass = Overpass()
nominatim = Nominatim()
cracow = nominatim.query('Kraków, Poland')
selectors = [['"tourism"="museum"'],
             ['"tourism"="gallery"'],
             ['"tourism"="aquarium"'],
             ['"tourism"="artwork"'],
             ['"tourism"="theme_park"'],
             ['"tourism"="viewpoint"'],
             ['"tourism"="zoo"'],
             ['"heritage"="2"'],
             ['"historic"="building"'],
             ['"historic"="castle"'],
             ['"historic"="city_gate"'],
             ['"leisure"="firepit"'],
             ['"leisure"="fishing"'],
             ['"leisure"="garden"'],
             ['"leisure"="water_park"'],
             ['"memorial"="statue"'],
             ['"military"="bunker"'],
             ['"natural"="peak"']]

v2 = db.get_collection('cracow-attractions-v2')
v4 = db.get_collection('cracow-attractions-v4')
for selector in selectors:
    query = overpassQueryBuilder(area=cracow.areaId(), elementType=['node', 'way'], selector=selector,
                                 out='body')
    res = overpass.query(query)
    count = 0
    for element in res.elements():
        osm = f"{element.type()}/{element.id()}"
        el = v2.find_one({"osm": osm})
        if el is None:
            count += 1
    print(count)
